In [1]:
# 모듈 로딩
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from bs4 import BeautifulSoup
from selenium import webdriver
from sklearn.utils import all_estimators
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from urllib.request import urlopen,Request,urlretrieve

ModuleNotFoundError: No module named 'cv2'

In [ ]:
# 동적 웹 크롤링으로 사진저장 함수
def image_save(urlpath,charname):
    driver = webdriver.Chrome('C:/Users/whrjs/PycharmProjects/exam/day_0725/chromedriver.exe')
    #driver = webdriver.Chrome('C:\\workspace’) # Windows 사용자의 설치 경로
    driver.get(urlpath)
    html=driver.page_source
    soup=BeautifulSoup(html,'html.parser')
    driver.quit()

    # 태그-속성 찾기
    image_finds=soup.select("div.bRMDJf img")
    
    # 이미지의 url찾기
    image=[]
    for image_find in image_finds:
        try:
            image.append(image_find['src'])
            image.append(image_find['data-src'])
        except:
            pass
    
    # 폴더 만들기
    if not os.path.exists('./'+charname):
        os.makedirs(charname)
    
    # 파일 저장하기
    for i in range(0,len(image)):
        urlretrieve(image[i],'./'+charname+'/'+charname+str(i)+'.png')

    # 파일 크기 조절
    for png in os.listdir('C:/machinelearning/project/'+charname):
        Image.open('C:/machinelearning/project/'+charname+'/'+png).resize((200,200)).save('./'+charname+'/'+png)